In [1]:
from pcg_gazebo.generators.creators import create_models_from_config
from pcg_gazebo.task_manager import Server
import os

# Start an empty world Gazebo simulation
server = Server()
server.create_simulation('default')
simulation = server.get_simulation('default')
simulation.create_gazebo_empty_world_task()
print(simulation.get_task_list())
print('Is Gazebo running: {}'.format(
    simulation.is_task_running('gazebo')))
simulation.run_all_tasks()

# Create a Gazebo proxy
gazebo_proxy = simulation.get_gazebo_proxy()

2021-01-01 13:33:43,679 | WARNING | __init__ | Error testing Gazebo server, message=Command '['rostopic', 'list']' returned non-zero exit status 1
['gazebo']
Is Gazebo running: False
2021-01-01 13:33:43,976 | WARNING | __init__ | Error testing roscore, message=Command '['rostopic', 'list']' returned non-zero exit status 1
2021-01-01 13:33:43,976 | WARNING | __init__ | Error testing Gazebo server, message=Command '['rostopic', 'list']' returned non-zero exit status 1
2021-01-01 13:33:44,273 | WARNING | __init__ | Error testing roscore, message=Command '['rostopic', 'list']' returned non-zero exit status 1
2021-01-01 13:33:44,559 | WARNING | __init__ | Error testing roscore, message=Command '['rostopic', 'list']' returned non-zero exit status 1
2021-01-01 13:33:44,774 | WARNING | __init__ | Error testing Gazebo server, message=Command '['rostopic', 'list']' returned non-zero exit status 1


In [2]:
import random

def create_and_spawn(config, pos=None):
    models = create_models_from_config(config)
    for model in models:
        model.spawn(
            gazebo_proxy=gazebo_proxy, 
            robot_namespace=model.name,
            pos=pos if pos is not None else [
                20 * random.random() - 10, 
                20 * random.random() - 10, 
                2 * random.random()
            ])

In [3]:
from pcg_gazebo.generators.shapes import random_rectangle, \
    random_points_to_triangulation, random_rectangles

config = [
    dict(
        type='extrude',
        args=dict(
            polygon=random_rectangle(),
            height=0.2,
            extrude_boundaries=False,
            name='extruded_poly_random_rectangle',
            color=None,
        )
    )
]

create_and_spawn(config, [-20, 0, 0.1])

config = [
    dict(
        type='extrude',
        args=dict(
            polygon=random_points_to_triangulation(),
            height=0.8,
            thickness=0.1,
            extrude_boundaries=True,
            name='extruded_poly_triangulation',
            color='random'
        )
    )
]

create_and_spawn(config, [0, 0, 0.4])

config = [
    dict(
        type='extrude',
        args=dict(
            polygon=random_rectangles(),
            height=0.5,
            thickness=0.15,
            extrude_boundaries=True,
            name='extruded_poly_walls',
            color='xkcd'
        )
    )
]

create_and_spawn(config, [20, 0, 0.25])

In [4]:
config = [
    dict(
        type='box_factory',
        args=dict(
            size=[
                [0.1, 0.4, 0.5],
                [1, 2, 3]
            ],
            name='box_static_var_size',
            use_permutation=True,
            color='xkcd'
        )
    ),
    dict(
        type='box_factory',
        args=dict(
            size=[
                [0.1, 0.4, 0.5],
                [1, 2, 3]
            ],
            mass=12,
            name='box_dynamic_var_size',
            use_permutation=False,
            color='xkcd'
        )
    ),
    dict(
        type='box_factory',
        args=dict(
            size=[
                [0.2, 0.4, 0.15],
                [1.2, 0.25, 0.7]
            ],
            mass=[5, 2],
            name='box_dynamic_permutate_size_mass',
            use_permutation=True,
            color='xkcd'
        )
    )
]

create_and_spawn(config)

In [5]:
config = [
    dict(
        type='box_factory',
        args=dict(
            size="__import__('numpy').random.random((2, 3))",
            use_permutation=True,
            name='box_static_lambdas',
            color='random'
        )
    ),
    dict(
        type='box_factory',
        args=dict(
            size="__import__('numpy').random.random((4, 3))",
            mass="__import__('numpy').arange(1, 10, 2)",
            use_permutation=True,
            name='box_dynamic_lambdas',
            color='random'
        )
    )
]

create_and_spawn(config)

In [6]:
config = [
    dict(
        type='cylinder',
        args=dict(
            radius=3,
            length=2, 
            mass=10,
            name='cylinder',
            pose=[0, 0, 1, 0, 0, 0]
        )
    ),
    dict(
        type='cylinder_factory',
        args=dict(
            length=[0.3, 0.5],
            radius=[0.2, 0.4],
            mass=[5, 2],
            name='cylinder_dynamic_permutate_radius_length_mass',
            use_permutation=True,
            color='xkcd'
        )
    ),
    dict(
        type='cylinder_factory',
        args=dict(
            length="__import__('numpy').linspace(0.1, 10, 2)",
            radius="__import__('numpy').random.random(2)",
            mass="__import__('numpy').arange(1, 2, 1)",
            use_permutation=True,
            name='cylinder_dynamic_lambdas',
            color='xkcd'
        )
    )
]

create_and_spawn(config)

In [7]:

config = [
    dict(
        type='sphere',
        args=dict(
            radius=3,
            mass=10,
            name='sphere',
            pose=[0, 0, 1.5, 0, 0, 0]
        )
    ),
    dict(
        type='sphere_factory',
        args=dict(
            radius=[0.3, 0.9],
            mass=[5, 2],
            name='sphere_dynamic_permutate_radius_mass',
            use_permutation=True,
            color='xkcd'
        )
    ),
    dict(
        type='sphere_factory',
        args=dict(
            radius="__import__('numpy').random.random(2) * 3",
            mass="__import__('numpy').arange(1, 4, 1)",
            use_permutation=True,
            name='sphere_dynamic_lambdas',
            color='xkcd'
        )
    )
]

create_and_spawn(config)

In [10]:
mesh_filename = os.path.abspath('./meshes/monkey_offset.stl')

config = [
    dict(
        type='mesh',
        args=dict(
            visual_mesh=mesh_filename,
            visual_mesh_scale=[1, 1, 1],
            use_approximated_collision=False,
            mass=10,
            name='monkey_dynamic_no_approx_collision',
            color='xkcd'
        )
    ),
    dict(
        type='mesh',
        args=dict(
            visual_mesh=mesh_filename,
            visual_mesh_scale=[1, 1, 1],
            use_approximated_collision=True,
            approximated_collision_model='box',
            mass=20,
            name='monkey_dynamic_with_approx_collision_box',
            color='xkcd'
        )
    ),
    dict(
        type='mesh',
        args=dict(
            visual_mesh=mesh_filename,
            visual_mesh_scale=[1, 1, 1],
            use_approximated_collision=True,
            mass=15,
            approximated_collision_model='cylinder',
            name='monkey_dynamic_with_approx_collision_cylinder',
            color='xkcd'
        )
    ),
    dict(
        type='mesh',
        args=dict(
            visual_mesh=mesh_filename,
            visual_mesh_scale=[1, 1, 1],
            use_approximated_collision=True,
            mass=3,
            approximated_collision_model='sphere',
            name='monkey_dynamic_with_approx_collision_sphere',
            color='xkcd'
        )
    ),
    dict(
        type='mesh',
        args=dict(
            visual_mesh=mesh_filename,
            visual_mesh_scale=[1, 1, 1],
            use_approximated_collision=True,
            mass=3,
            approximated_collision_model='sphere',
            name='monkey_dynamic_defined_inertia',
            color='xkcd',
            use_approximated_inertia=False,
            inertia=dict(
                ixx=0.1,
                iyy=0.1,
                izz=0.1
            )                 
        )
    )
]

create_and_spawn(config)

AssertionError: Filename could not be resolved from /root/pcg_tameshi/meshes/monkey_offset.stl

In [11]:
simulation.kill_all_tasks()

2021-01-01 13:45:20,895 | WARNING | task | Task <roscore> - Killing process tree...
2021-01-01 13:45:20,904 | WARNING | task | Sending SIGINT to child process id=20618
2021-01-01 13:45:20,906 | WARNING | task | Child process 20618 still running
2021-01-01 13:45:20,907 | WARNING | task | Sending SIGINT to child process id=20661
2021-01-01 13:45:20,908 | WARNING | task | Child process 20661 still running
2021-01-01 13:45:20,909 | WARNING | task | Sending SIGINT to child process id=20713
2021-01-01 13:45:20,910 | WARNING | task | Child process 20713 still running
2021-01-01 13:45:20,911 | WARNING | task | Sending SIGINT to child process id=20617
2021-01-01 13:45:20,911 | WARNING | task | Child process 20617 still running
2021-01-01 13:45:21,171 | WARNING | task | Process 20713 already terminated
2021-01-01 13:45:21,174 | WARNING | task | Process 20661 already terminated
2021-01-01 13:45:21,312 | WARNING | task | Process 20617 already terminated
2021-01-01 13:45:21,314 | WARNING | task | P